In [ ]:
from tsfresh.examples.robot_execution_failures import load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

# Load and process data

In [ ]:
df, y = load_robot_execution_failures()
df.head()

In [ ]:
X = extract_features(df, column_id='id', column_sort='time')

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X_filtered = extract_relevant_features(df, y, column_id='id', column_sort='time')

In [ ]:
X_filtered.head()

In [ ]:
X_filtered.info()

In [ ]:
X_train, X_test, X_filtered_train, X_filtered_test, y_train, y_test = train_test_split(X, X_filtered, y, test_size=.4)

In [ ]:
# Train and evaluate classifier

In [ ]:
cl = DecisionTreeClassifier()
cl.fit(X_train, y_train)
print(classification_report(y_test, cl.predict(X_test)))

In [ ]:
cl.n_features_

In [ ]:
cl2 = DecisionTreeClassifier()
cl2.fit(X_filtered_train, y_train)
print(classification_report(y_test, cl2.predict(X_filtered_test)))

In [ ]:
cl2.n_features_

In [ ]:
# Plot some selected algorithms

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
id_curr=3

plt.figure(figsize=[12, 6])
plt.suptitle("Time series for id={} (no failure)".format(id_curr), fontsize=18)

plt.subplot(2,3,1)
plt.plot(range(15), df.loc[df.id==id_curr, "a"])
plt.ylim([-30, 30])
plt.title("a")

plt.subplot(2,3,2)
plt.plot(range(15), df.loc[df.id==id_curr, "b"])
plt.ylim([-30, 30])
plt.title("b")

plt.subplot(2,3,3)
plt.plot(range(15), df.loc[df.id==id_curr, "c"])
plt.ylim([0, 100])
plt.title("c")

plt.subplot(2,3,4)
plt.plot(range(15), df.loc[df.id==id_curr, "d"])
plt.ylim([-30, 30])
plt.title("d")

plt.subplot(2,3,5)
plt.plot(range(15), df.loc[df.id==id_curr, "e"])
plt.ylim([-30, 30])
plt.title("e")

plt.subplot(2,3,6)
plt.plot(range(15), df.loc[df.id==id_curr, "f"])
plt.ylim([-30, 30])
plt.title("f")

plt.savefig("ts_example_robot_failures_nofail.png")
plt.show()

In [ ]:
id_curr=20

plt.figure(figsize=[12, 6])
plt.suptitle("Time series for id={} (failure)".format(id_curr), fontsize=18)

plt.subplot(2,3,1)
plt.plot(range(15), df.loc[df.id==id_curr, "a"])
plt.ylim([-30, 30])
plt.title("a", fontsize=17)

plt.subplot(2,3,2)
plt.plot(range(15), df.loc[df.id==id_curr, "b"])
plt.ylim([-30, 30])
plt.title("b")

plt.subplot(2,3,3)
plt.plot(range(15), df.loc[df.id==id_curr, "c"])
plt.ylim([0, 100])
plt.title("c")

plt.subplot(2,3,4)
plt.plot(range(15), df.loc[df.id==id_curr, "d"])
plt.ylim([-30, 30])
plt.title("d")

plt.subplot(2,3,5)
plt.plot(range(15), df.loc[df.id==id_curr, "e"])
plt.ylim([-30, 30])
plt.title("e")

plt.subplot(2,3,6)
plt.plot(range(15), df.loc[df.id==id_curr, "f"])
plt.ylim([-30, 30])
plt.title("f")

plt.savefig("ts_example_robot_failures_fail.png")
plt.show()

In [ ]:
plt.close()